# 🏦 Pipeline de Ingeniería de Datos: Morosidad Bancaria Perú (2018-2025)

Autor: Croszer Luza Paredes
Herramientas: Python, Pandas, Glob, OS.
Fuente de Datos: Superintendencia de Banca, Seguros y AFP (SBS).

## 📋 Descripción del Proyecto
Este script automatiza la extracción, transformación y carga (ETL) de información financiera pública. El objetivo es consolidar más de **90 reportes mensuales** de morosidad (archivos Excel no estructurados y con formatos cambiantes) en un Modelo de Datos Analítico listo para Power BI.

##
⚙️ Desafíos Técnicos Resueltos
1.  **Lectura Dinámica:** Iteración sobre archivos con nombres no estandarizados.
2.  **Limpieza de Texto:** Normalización de nombres de entidades bancarias (ej. "BBVA Continental" vs "BBVA").
3.  **Corrección de Tipos:** Conversión de columnas numéricas con formatos mixtos (texto/número).
4.  **Estandarización:** Creación de categorías de riesgo homogéneas a lo largo de 7 años.

---

In [50]:
#
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive montado correctamente")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado correctamente


In [51]:
sample_limit=10
sample_limit=None

In [53]:
import os, re, math
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob

In [54]:
# Rutas del drive
FOLDER  = "/content/drive/MyDrive/trabajo SBS/SBS_Morosidad_Mensual"
OUT_DIR = "/content/drive/MyDrive/trabajo SBS/parsed_repair"
os.makedirs(OUT_DIR, exist_ok=True)

HEADER_KEYWORDS = [
    'concepto','créditos','creditos','concept','total',
    'entidad','empresa','credito','morosidad','periodo'
]


In [55]:

# DICCIONARIOS DE ESTANDARIZACIÓN
# Fusiones y cambios de nombre conocidos en el sistema bancario peruano
ENTIDADES_MAPEO = {
    'CITIBANK'                          : 'SCOTIABANK (EX-CITIBANK)',
    'BANCO INTERAMERICANO DE FINANZAS'  : 'SCOTIABANK (EX-BIF)',
    'BIF'                               : 'SCOTIABANK (EX-BIF)',
    'BANCO FINANCIERO'                  : 'BANCO PICHINCHA (EX-FINANCIERO)',
    'HSBC BANK PERU'                    : 'GNB SUDAMERIS (EX-HSBC)',
    'DEUTSCHE BANK PERU'                : 'BANCO DE COMERCIO (EX-DEUTSCHE)',
}

# Estandarizamos por  de tipos de crédito
TIPOS_CREDITO_KEYWORDS = {
    'CORPORATIVOS'      : ['corporativo', 'corp'],
    'GRANDES EMPRESAS'  : ['grande empresa', 'grandes empresas', 'gran empresa'],
    'MEDIANAS EMPRESAS' : ['mediana empresa', 'medianas empresas'],
    'PEQUEÑAS EMPRESAS' : ['pequeña empresa', 'pequeñas empresas', 'pequena'],
    'MICROEMPRESAS'     : ['microempresa', 'micro empresa', 'microempresas'],
    'CONSUMO'           : ['consumo'],
    'HIPOTECARIO'       : ['hipotecario', 'hipotecaria', 'mivivienda', 'vivienda'],
    'TOTAL'             : ['total créditos', 'total creditos', 'total directo'],
}


In [57]:
#  ORDENO ESTOS ARCHIVOS POR FECHA
MONTHS_ORDER = {
    'en':1, 'fe':2, 'ma':3, 'ab':4, 'my':5, 'jn':6,
    'jl':7, 'ag':8, 'se':9, 'oc':10,'no':11,'di':12
}

MONTHS_MAP_FULL = {
    'ene':1,'en':1, 'feb':2,'fe':2, 'mar':3,'ma':3,
    'abr':4,'ab':4, 'may':5,'my':5, 'jun':6,'jn':6,
    'jul':7,'jl':7, 'ago':8,'ag':8, 'sep':9,'se':9,
    'set':9,'oct':10,'oc':10,'nov':11,'no':11,'dic':12,'di':12
}

def get_sort_key(path):
    """Extrae (año, mes) del nombre de archivo para ordenar cronológicamente"""
    stem = Path(path).stem.lower()
    m = re.search(r'([a-z]{2,3})(\d{4})', stem)
    if m:
        mes_code = m.group(1)
        anio     = int(m.group(2))
        mes_num  = MONTHS_MAP_FULL.get(mes_code[:3], 0) or MONTHS_MAP_FULL.get(mes_code[:2], 0)
        return (anio, mes_num)
    return (9999, 99)

def list_excel_files(folder):
    """Lista archivos Excel ordenados CRONOLÓGICAMENTE"""
    exts = ('*.xls', '*.XLS', '*.xlsx', '*.XLSX')
    files = []
    for e in exts:
        files.extend(glob(os.path.join(folder, e)))
    # Ordenar por fecha mas no alfabéticamente
    files = sorted(files, key=get_sort_key)
    return files


In [58]:
# FUNCIONES DE LECTURA Y DETECCIÓN DE ESTRUCTURA
def read_raw_excel(path):
    """Lee el Excel sin header para ver la estructura cruda"""
    try:
        df = pd.read_excel(path, header=None, dtype=object, engine='xlrd')
    except Exception:
        df = pd.read_excel(path, header=None, dtype=object, engine='openpyxl')
    return df

def is_textual_row(row):
    """Heurística: ¿es esta fila principalmente texto?"""
    non_null = [x for x in row if not (x is None or (isinstance(x, float) and math.isnan(x)))]
    text_count = sum(
        1 for v in non_null
        if isinstance(v, str) and re.search(r'[A-Za-zÁÉÍÓÚáéíóúñÑ]', v)
    )
    return text_count >= max(1, int(len(non_null) * 0.4))

def detect_header_row(raw):
    """Detecta en qué fila empiezan los encabezados de columna"""
    maxrows = min(80, raw.shape[0])
    for i in range(maxrows):
        row_str = raw.iloc[i].astype(str).str.lower().fillna('')
        joined  = ' '.join(row_str.values)
        if any(k in joined for k in HEADER_KEYWORDS):
            return i
        if is_textual_row(raw.iloc[i].tolist()) and i < 30:
            return i
    for i in range(maxrows):
        if raw.iloc[i].astype(str).str.strip()\
               .map(lambda x: bool(re.search(r'[A-Za-z]', str(x)))).sum() >= 3:
            return i
    return 0

def build_colnames_from_multiheader(raw, header_row, max_header_rows=5):
    """
    Junta varias filas de encabezado para formar nombres de columna únicos.
    dE reportes SBS
    """
    last = min(header_row + max_header_rows, raw.shape[0])
    header_block = raw.iloc[header_row:last].fillna('').astype(str)

    cols = []
    for c in range(header_block.shape[1]):
        parts = [
            header_block.iat[r, c].strip()
            for r in range(header_block.shape[0])
            if header_block.iat[r, c].strip() != ''
        ]
        cols.append(' | '.join(parts).strip())
    return cols, header_block.shape[0]


In [59]:

# NORMALIZACIÓN NUMÉRICA
def normalize_numeric_str(s):
    """
    Convierte cualquier representación de número a float.
    Maneja: puntos, comas europeas y americanas, paréntesis como negativos, %.
    """
    if pd.isna(s):
        return None
    s = str(s).strip()
    if s == '':
        return None

    s = re.sub(r'[^\d\-,\.()%−– ]', '', s)

    # Paréntesis = negativo
    if '(' in s and ')' in s:
        s = s.replace('(', '-').replace(')', '')

    s = s.replace('%', '').replace('\xa0', '').replace(' ', '')

    # Detectar separadores decimales
    if s.count('.') > 0 and s.count(',') > 0:
        s = s.replace('.', '').replace(',', '.')   # europeo: 1.234,56
    elif s.count(',') > 1 and s.count('.') == 0:
        s = s.replace(',', '')                     # miles con coma
    elif s.count(',') == 1 and s.count('.') == 0:
        s = s.replace(',', '.')                    # decimal con coma

    try:
        return float(s)
    except:
        return None


In [60]:
# ESTANDARIZACIÓN DE ENTIDADES Y TIPOS DE CRÉDITO
def estandarizar_entidad(nombre_raw):
    if pd.isna(nombre_raw):
        return 'DESCONOCIDO'
    nombre_upper = str(nombre_raw).upper().strip()
 # Limpiamos caracteres de salto de línea frecuentes en celdas Excel
    nombre_upper = nombre_upper.replace('\n', ' ').replace('\r', ' ')
    nombre_upper = re.sub(r'\s+', ' ', nombre_upper).strip()

    for original, estandar in ENTIDADES_MAPEO.items():
        if original.upper() in nombre_upper:
            return estandar
    return nombre_upper

def clasificar_tipo_credito(concepto, subgrupo, tipo_raw):
    texto = f"{concepto} {subgrupo} {tipo_raw}".lower()
    for tipo_std, keywords in TIPOS_CREDITO_KEYWORDS.items():
        for kw in keywords:
            if kw in texto:
                return tipo_std
    if 'total' in texto:
        return 'TOTAL'
    return 'OTROS'

def clasificar_tipo_metrica(concepto):
    """
    CRÍTICO: Determina si el valor es un RATIO (%) o un SALDO (S/.)
    Los ratios se promedian, los saldos se suman.
    """
    cl = str(concepto).lower()
    if any(x in cl for x in ['ratio', 'morosidad', 'tasa', '%', 'porcentaje', 'índice', 'indice']):
        return 'RATIO'
    if any(x in cl for x in ['saldo', 'crédito', 'credito', 'cartera', 'monto',
                              'vencido', 'vigente', 'judicial', 'refinanciado',
                              'reestructurado', 'colocacion']):
        return 'SALDO'
    return 'OTRO'


In [61]:
# PARSEO DE UN ARCHIVO INDIVIDUAL

def parse_single_excel(path, debug=False):
    raw = read_raw_excel(path)
    if debug:
        print(f"  raw shape: {raw.shape}")

    header_row              = detect_header_row(raw)
    colnames, n_header_rows = build_colnames_from_multiheader(raw, header_row, max_header_rows=5)

    # Datos debajo del bloque de encabezados
    data = raw.iloc[header_row + n_header_rows:].copy().reset_index(drop=True)
    data.columns = colnames[:data.shape[1]]

    # Detectamos cuántas columnas de la izquierda son etiquetas como concepto, sub, tipo
    left_candidate = min(3, data.shape[1])
    label_cols = []
    for c in range(left_candidate):
        score = data.iloc[:, c].astype(str).fillna('').map(
            lambda x: bool(re.search(r'[A-Za-zÁÉÍÓÚáéíóúñÑ]', x))
        ).sum()
        if score >= int(0.25 * len(data)):
            label_cols.append(c)
    if not label_cols:
        label_cols = [0]

    # Construimos etiquetas jerárquicas
    def make_labels(row):
        vals = [str(row.iloc[i]).strip() for i in label_cols]
        vals = [v for v in vals if v not in ('', 'nan')]
        concepto = vals[0] if len(vals) > 0 else ''
        subgrupo = vals[1] if len(vals) > 1 else ''
        tipo_raw = vals[2] if len(vals) > 2 else ''
        return concepto, subgrupo, tipo_raw

    labels = data.apply(make_labels, axis=1, result_type='expand')
    labels.columns = ['concepto', 'subgrupo', 'tipo_credito_raw']

    # Columnas de entidades bancarias
    bank_col_names = [c for i, c in enumerate(data.columns) if i not in label_cols]
    data_banks     = data[bank_col_names].copy()

    # ---- Extraer período del nombre del archivo ----
    stem    = Path(path).stem.lower()
    m       = re.search(r'([a-z]{2,4})(\d{4})', stem)
    periodo = pd.NaT
    if m:
        mon_code = m.group(1)
        yr       = int(m.group(2))
        mo       = MONTHS_MAP_FULL.get(mon_code[:3]) or MONTHS_MAP_FULL.get(mon_code[:2])
        if mo:
            periodo = pd.Timestamp(year=yr, month=mo, day=1)

    # ---- Convertir a formato largo ----
    src       = Path(path).name
    long_rows = []

    for idx in range(data_banks.shape[0]):
        conc    = labels.iat[idx, 0]
        sub     = labels.iat[idx, 1]
        tip_raw = labels.iat[idx, 2]

        for col in data_banks.columns:
            num = normalize_numeric_str(data_banks.at[idx, col])
            if num is None:
                continue

            long_rows.append({
                'source_file'      : src,
                'periodo'          : periodo,
                'concepto'         : conc,
                'subgrupo'         : sub,
                'tipo_credito_raw' : tip_raw,
                'entidad_raw'      : str(col).strip(),
                'valor_num'        : num,
            })

    df_long = pd.DataFrame(long_rows)

    # ---- Aplicar estandarizaciones ----
    df_long['entidad_std']      = df_long['entidad_raw'].apply(estandarizar_entidad)
    df_long['tipo_credito_std'] = df_long.apply(
        lambda x: clasificar_tipo_credito(x['concepto'], x['subgrupo'], x['tipo_credito_raw']),
        axis=1
    )
    df_long['tipo_metrica'] = df_long['concepto'].apply(clasificar_tipo_metrica)

    return df_long


In [62]:
# PROCESAMIENTO EN LOTE
def reparse_all(folder, out_dir, sample_limit=None, debug=False):

    files = list_excel_files(folder)
    if sample_limit:
        files = files[:sample_limit]

    total    = len(files)
    all_dfs  = []
    failures = []

    print(f"📦 Procesando {total} archivos en orden cronológico...\n")

    for idx, f in enumerate(files, 1):
        nombre = Path(f).name
        try:
            dfp = parse_single_excel(f, debug=debug)
            dfp['source_path'] = f
            all_dfs.append(dfp)

            # Guardamos CSV individual en parsed_repair
            stem = Path(f).stem
            dfp.to_csv(os.path.join(out_dir, f"{stem}_parsed.csv"), index=False, encoding='utf-8-sig')

            print(f"  ✅ [{idx:02d}/{total}] {nombre:30s} → {len(dfp):>5,} filas | "
                  f"{dfp['periodo'].iloc[0].strftime('%Y-%m') if not dfp.empty and pd.notna(dfp['periodo'].iloc[0]) else '????-??'}")

        except Exception as e:
            print(f"  ❌ [{idx:02d}/{total}] {nombre} → ERROR: {e}")
            failures.append((nombre, str(e)))

    if not all_dfs:
        print("\n⚠️  No se parseó ningún archivo. Revisa la ruta FOLDER.")
        return None, failures

    # -- Consolidamos
    df_all = pd.concat(all_dfs, ignore_index=True, sort=False)

    # Limpiamos texto
    for col in ['entidad_raw', 'entidad_std', 'concepto', 'subgrupo',
                'tipo_credito_raw', 'tipo_credito_std']:
        if col in df_all.columns:
            df_all[col] = df_all[col].astype(str).str.replace('\n', ' ').str.strip()

    # Guardamos LONG completo
    out_long = os.path.join(out_dir, "sbs_long_COMPLETO.csv")
    df_all.to_csv(out_long, index=False, encoding='utf-8-sig')
    print(f"\n✅ sbs_long_COMPLETO.csv   → {df_all.shape[0]:,} filas")

    #  AGREGACIÓN CORRECTA Ratios vs Saldos por separado

    GROUP_KEYS = ['periodo', 'entidad_std', 'tipo_credito_std', 'concepto']

    df_ratios = df_all[df_all['tipo_metrica'] == 'RATIO'].copy()
    df_saldos = df_all[df_all['tipo_metrica'] == 'SALDO'].copy()
    df_otros  = df_all[df_all['tipo_metrica'] == 'OTRO'].copy()

    partes = []

    if not df_ratios.empty:
        agg_r = df_ratios.groupby(GROUP_KEYS, dropna=False)['valor_num'].mean().reset_index()
        agg_r['tipo_metrica'] = 'RATIO'
        agg_r.to_csv(os.path.join(out_dir, "sbs_agg_RATIOS.csv"), index=False, encoding='utf-8-sig')
        partes.append(agg_r)
        print(f"✅ sbs_agg_RATIOS.csv      → {len(agg_r):,} filas  (promedio ✓)")

    if not df_saldos.empty:
        agg_s = df_saldos.groupby(GROUP_KEYS, dropna=False)['valor_num'].sum().reset_index()
        agg_s['tipo_metrica'] = 'SALDO'
        agg_s.to_csv(os.path.join(out_dir, "sbs_agg_SALDOS.csv"), index=False, encoding='utf-8-sig')
        partes.append(agg_s)
        print(f"✅ sbs_agg_SALDOS.csv      → {len(agg_s):,} filas  (suma ✓)")

    if not df_otros.empty:
        agg_o = df_otros.groupby(GROUP_KEYS, dropna=False)['valor_num'].mean().reset_index()
        agg_o['tipo_metrica'] = 'OTRO'
        partes.append(agg_o)

    if partes:
        agg_todo = pd.concat(partes, ignore_index=True)
        agg_todo.to_csv(os.path.join(out_dir, "sbs_agg_COMBINADO.csv"), index=False, encoding='utf-8-sig')
        print(f"✅ sbs_agg_COMBINADO.csv   → {len(agg_todo):,} filas")

    return df_all, failures


In [63]:
# EJECUCIÓN PRINCIPAL
# Montar Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    print("(No está en COlab ?error)")

print("=" * 70)
print("PROCESAMIENTO SBS - VERSIÓN FINAL CORREGIDA")
print("=" * 70)

# Verificar carpeta
files_preview = list_excel_files(FOLDER)
print(f"\n📂 Carpeta: {FOLDER}")
print(f"✅ Archivos encontrados: {len(files_preview)}")

print("\n📋 Primeros 8 archivos (orden cronológico):")
for f in files_preview[:8]:
    print(f"   - {Path(f).name}")

print()

# ============================================================
# EJECUTAR
# Cambia sample_limit=10 para probar solo 10 archivos
# Cambia a sample_limit=None para procesar los 95
# ============================================================
df_all, failures = reparse_all(
    folder       = FOLDER,
    out_dir      = OUT_DIR,
    sample_limit = None,   # ← None = TODOS los archivos
    debug        = False
)

# ---- Resumen final ----
print("\n" + "=" * 70)
if df_all is not None:
    periodo_min = df_all['periodo'].min()
    periodo_max = df_all['periodo'].max()
    print("✅  RESUMEN FINAL")
    print("=" * 70)
    print(f"  Total filas          : {df_all.shape[0]:,}")
    print(f"  Período              : {periodo_min.strftime('%B %Y')} → {periodo_max.strftime('%B %Y')}")
    print(f"  Entidades únicas     : {df_all['entidad_std'].nunique()}")
    print(f"  Tipos de crédito     : {df_all['tipo_credito_std'].nunique()}")
    print(f"  Archivos procesados  : {df_all['source_file'].nunique()}/{len(files_preview)}")
    print(f"  Filas como RATIO     : {(df_all['tipo_metrica']=='RATIO').sum():,}")
    print(f"  Filas como SALDO     : {(df_all['tipo_metrica']=='SALDO').sum():,}")
    print(f"\n  📁 Archivos en: {OUT_DIR}")
    print(f"     - sbs_long_COMPLETO.csv")
    print(f"     - sbs_agg_RATIOS.csv")
    print(f"     - sbs_agg_SALDOS.csv")
    print(f"     - sbs_agg_COMBINADO.csv")
    print(f"     - [nombre]_parsed.csv (uno por cada Excel)")

if failures:
    print(f"\n⚠️  Archivos con error ({len(failures)}):")
    for nombre, err in failures:
        print(f"     ❌ {nombre}: {err}")

print("\n✨ Listo! Importa los CSVs a Power BI.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROCESAMIENTO SBS - VERSIÓN FINAL CORREGIDA

📂 Carpeta: /content/drive/MyDrive/trabajo SBS/SBS_Morosidad_Mensual
✅ Archivos encontrados: 95

📋 Primeros 8 archivos (orden cronológico):
   - B-2362-en2018.XLS
   - B-2362-fe2018.XLS
   - B-2362-ma2018.XLS
   - B-2362-ab2018.XLS
   - B-2362-my2018.XLS
   - B-2362-jn2018.XLS
   - B-2362-jl2018.XLS
   - B-2362-ag2018.XLS

📦 Procesando 95 archivos en orden cronológico...

  ✅ [01/95] B-2362-en2018.XLS              →   512 filas | 2018-01
  ✅ [02/95] B-2362-fe2018.XLS              →   516 filas | 2018-02
  ✅ [03/95] B-2362-ma2018.XLS              →   513 filas | 2018-03
  ✅ [04/95] B-2362-ab2018.XLS              →   511 filas | 2018-04
  ✅ [05/95] B-2362-my2018.XLS              →   508 filas | 2018-05
  ✅ [06/95] B-2362-jn2018.XLS              →   510 filas | 2018-06
  ✅ [07/95] B-2362-jl2018.XLS              →   510

PREPARACIÓN PARA POWER BI (MODELADO DE DATOS)



In [64]:
import pandas as pd
import numpy as np
import os

In [65]:
INPUT_FILE  = "/content/drive/MyDrive/trabajo SBS/parsed_repair/sbs_long_COMPLETO.csv"
OUTPUT_DIR  = "/content/drive/MyDrive/trabajo SBS/powerbi_final"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Cargado archivo ")
df = pd.read_csv(INPUT_FILE)

Cargado archivo 


In [66]:
# 2. CORRECCIÓN CRÍTICA DE NEGOCIO

In [67]:
# 3. LIMPIEZA FINAL DE ENTIDADES
# Esto dejará los nombres listos para presentar por Ejemplo: "B. CONTINENTAL" -> "BBVA"
mapeo_final_bancos = {
    'B. CONTINENTAL': 'BBVA',
    'B. BBVA PERÚ': 'BBVA',
    'BANCO BBVA PERÚ': 'BBVA',
    'B. DE CRÉDITO DEL PERÚ (CON SUCURSALES EN EL EXTERIOR)': 'BCP',
    'B. DE CRÉDITO DEL PERÚ': 'BCP',
    'BANCO DE CRÉDITO DEL PERÚ': 'BCP',
    'B. FINANCIERO': 'Pichincha',
    'B. PICHINCHA': 'Pichincha',
    'BANCO PICHINCHA': 'Pichincha',
    'SCOTIABANK PERÚ': 'Scotiabank',
    'INTERBANK (CON SUCURSALES EN EL EXTERIOR)': 'Interbank',
    'INTERBANK': 'Interbank',
    'MIBANCO': 'Mibanco',
    'B. INTERNACIONAL DEL PERÚ': 'Interbank',
    'CITIBANK': 'Citibank',  # Banca corporativa
    'B. DE COMERCIO': 'Bancom', # Cambio de nombre reciente
    'BANCO DE COMERCIO': 'Bancom',
    'B. GNB': 'GNB',
    'B. FALABELLA PERÚ': 'Falabella',
    'B. RIPLEY': 'Ripley',
    'B. SANTANDER PERÚ': 'Santander',
    'B. AZTECA PERÚ': 'Azteca',
    'ALFIN BANCO': 'Alfin', # Antes Azteca en algunos contextos o nuevo
    'B. ICBC': 'ICBC',
    'BANK OF CHINA': 'Bank of China',
    'B. CENCOSUD': 'Cencosud',
}

In [68]:
# Aplicamos el mapeo sobre la columna estandarizada (asegurando mayúsculas)
df['entidad_std_clean'] = df['entidad_std'].str.upper().str.strip()
df['entidad_final'] = df['entidad_std_clean'].map(mapeo_final_bancos).fillna(df['entidad_std'])

#

In [69]:
# CREACIÓN DE TABLAS DIMENSIONALES

In [70]:
# Solo nos quedamos con las columnas necesarias
cols_fact = ['periodo', 'entidad_final', 'tipo_credito_std', 'valor_num', 'source_file']
df_fact = df[cols_fact].copy()
df_fact.rename(columns={'valor_num': 'morosidad_ratio', 'entidad_final': 'entidad'}, inplace=True)

In [71]:
df_fact['morosidad_ratio_decimal'] = df_fact['morosidad_ratio'] / 100

In [72]:
#Dimensión Calendario
fechas_unicas = df_fact['periodo'].unique()
dim_calendario = pd.DataFrame({'Fecha': fechas_unicas})
dim_calendario['Fecha'] = pd.to_datetime(dim_calendario['Fecha'])
dim_calendario['Año'] = dim_calendario['Fecha'].dt.year
dim_calendario['Mes'] = dim_calendario['Fecha'].dt.month
dim_calendario['Mes_Nombre'] = dim_calendario['Fecha'].dt.strftime('%B')
dim_calendario = dim_calendario.sort_values('Fecha')

In [73]:
#Dimensión Entidad
entidades_unicas = df_fact['entidad'].unique()
dim_entidad = pd.DataFrame({'Entidad': entidades_unicas})
dim_entidad = dim_entidad.sort_values('Entidad')

In [74]:
#5. EXPORTACIÓN
print(f"Exportando archivos a: {OUTPUT_DIR}")

Exportando archivos a: /content/drive/MyDrive/trabajo SBS/powerbi_final


In [75]:
# Archivo Principal
df_fact.to_csv(os.path.join(OUTPUT_DIR, "Fact_Morosidad_SBS.csv"), index=False, encoding='utf-8-sig')

# Dimensiones
dim_calendario.to_csv(os.path.join(OUTPUT_DIR, "Dim_Calendario.csv"), index=False, encoding='utf-8-sig')
dim_entidad.to_csv(os.path.join(OUTPUT_DIR, "Dim_Entidad.csv"), index=False, encoding='utf-8-sig')

print("✅ ¡Proceso completado! Archivos listos para Power BI.")
print(f"Total registros procesados: {len(df_fact)}")
print(df_fact.head())

✅ ¡Proceso completado! Archivos listos para Power BI.
Total registros procesados: 48639
      periodo                         entidad tipo_credito_std  \
0  2018-01-01                            BBVA     CORPORATIVOS   
1  2018-01-01                             BCP     CORPORATIVOS   
2  2018-01-01                       Pichincha     CORPORATIVOS   
3  2018-01-01  B. INTERAMERICANO DE FINANZAS*     CORPORATIVOS   
4  2018-01-01                      Scotiabank     CORPORATIVOS   

   morosidad_ratio        source_file  morosidad_ratio_decimal  
0         0.001034  B-2362-en2018.XLS                 0.000010  
1         0.071689  B-2362-en2018.XLS                 0.000717  
2         0.391574  B-2362-en2018.XLS                 0.003916  
3         0.283066  B-2362-en2018.XLS                 0.002831  
4         0.057789  B-2362-en2018.XLS                 0.000578  


In [76]:
import pandas as pd
import os

# 1. Cargar la data 'sucia'
df = pd.read_csv("/content/drive/MyDrive/trabajo SBS/powerbi_final/Fact_Morosidad_SBS.csv")

In [77]:
# 2. Diccionario de Limpieza (Mapeo Definitivo)
# Este diccionario arregla todos los errores que encontré en tu data
correcciones = {
    # Unificar Pichincha
    'B. PICHINCHA*': 'Pichincha',
    'B. FINANCIERO': 'Pichincha',

    # Unificar Bancom (antes Banco de Comercio)
    'BANCOM': 'Bancom',
    'B. DE COMERCIO': 'Bancom',
    'BANCO DE COMERCIO': 'Bancom',

    # Unificar Alfin (antes Azteca)
    'ALFIN BANCO1/': 'Alfin',
    'ALFIN BANCO': 'Alfin',
    'AZTECA': 'Alfin', # Para ver la historia continua
    'B. AZTECA PERÚ': 'Alfin',

    # Unificar Compartamos
    'COMPARTAMOS BANCO*': 'Compartamos',
    'COMPARTAMOS BANCO': 'Compartamos',

    # Unificar BanBif
    'B. INTERAMERICANO DE FINANZAS': 'BanBif',
    'B. INTERAMERICANO DE FINANZAS*': 'BanBif',

    # Unificar BBVA
    'B. BBVA PERÚ*': 'BBVA',
    'B. CONTINENTAL': 'BBVA',

    # Unificar Interbank
    'INTERBANK (CON SUCURSALES EN EL EXTERIOR)**': 'Interbank',
    'INTERBANK': 'Interbank',

    # Unificar BCI
    'B. BCI PERÚ': 'BCI',
    'B. BCI PERÚ**': 'BCI',

    # Unificar Scotiabank
    'SCOTIABANK (EX-CITIBANK)': 'Scotiabank', # Opcional: únelo si quieres ver todo el banco junto
    'SCOTIABANK PERÚ': 'Scotiabank',

    # Estandarizar Falabella/Ripley/GNB (Quitar 'B.' si existe o dejar limpio)
    'B. FALABELLA PERÚ': 'Falabella',
    'B. RIPLEY': 'Ripley',
    'B. GNB': 'GNB'
}

In [78]:
#  Aplicamos Correcciones
# Primero convertimos todo a string para evitar errores
df['entidad'] = df['entidad'].astype(str).str.strip()

# Aplicamos el mapeo. Si no está en el diccionario, se queda como estaba
df['entidad'] = df['entidad'].replace(correcciones)

# Forzamos Title Case  para arreglar "BANCOM" vs "Bancom"
# Excepción: Siglas como BBVA, BCP, GNB, ICBC
siglas = ['BBVA', 'BCP', 'GNB', 'ICBC', 'BCI']
df['entidad'] = df['entidad'].apply(lambda x: x.upper() if x.upper() in siglas else x.title())

# Caso especial manual por si acaso
df.loc[df['entidad'].str.upper() == 'BANCOM', 'entidad'] = 'Bancom'
df.loc[df['entidad'].str.upper() == 'ALFIN', 'entidad'] = 'Alfin'

In [79]:
# 4. Guardar Archivos Limpios
print("Guardando archivos corregidos...")
df.to_csv("/content/drive/MyDrive/trabajo SBS/powerbi_final/Fact_Morosidad_SBS_Clean.csv", index=False)

# Regenerar Dim_Entidad limpia
entidades_unicas = sorted(df['entidad'].unique())
dim_entidad_clean = pd.DataFrame({'entidad': entidades_unicas})
dim_entidad_clean.to_csv("/content/drive/MyDrive/trabajo SBS/powerbi_final/Dim_Entidad_Clean.csv", index=False)

print("¡Listo! Usa 'Fact_Morosidad_SBS_Clean.csv' y 'Dim_Entidad_Clean.csv' en Power BI.")
print("Entidades finales:", entidades_unicas)


Guardando archivos corregidos...
¡Listo! Usa 'Fact_Morosidad_SBS_Clean.csv' y 'Dim_Entidad_Clean.csv' en Power BI.
Entidades finales: ['Alfin', 'Azteca', 'BBVA', 'BCI', 'BCP', 'Banbif', 'Bancom', 'Bank Of China', 'Cencosud', 'Compartamos', 'Falabella', 'GNB', 'ICBC', 'Interbank', 'Mibanco', 'Pichincha', 'Ripley', 'Santander', 'Santander Consumer Bank', 'Scotiabank', 'Total Banca Múltiple']


MEJORAMOS DIM_ENTIDAD_CLEAN PARA QUE POWER BI LO LEA MEJOR


In [82]:
import pandas as pd

# 1. Cargamos tu lista actual
df_entidades = pd.read_csv("/content/drive/MyDrive/trabajo SBS/powerbi_final/Dim_Entidad_Clean.csv")

# 2. Creamos categorías para que tus gráficos en Power BI sean mejores
def categorizar_banco(nombre):
    nombre = nombre.lower()
    if nombre in ['bcp', 'bbva', 'scotiabank', 'interbank']:
        return 'Bancos Top 4'
    elif nombre in ['mibanco', 'compartamos']:
        return 'Microfinanzas'
    elif nombre in ['falabella', 'ripley', 'cencosud', 'santander consumer bank']:
        return 'Retail / Consumo'
    elif 'total' in nombre:
        return 'Agregado Sistémico'
    else:
        return 'Banca Múltiple / Otros'

# 3. Construimos la tabla pro
df_entidades['Entidad_ID'] = range(1, len(df_entidades) + 1)  # Un ID numérico
df_entidades['Tipo_Entidad'] = df_entidades['entidad'].apply(categorizar_banco)  # <- corregido

# Reordenamos columnas
df_entidades = df_entidades[['Entidad_ID', 'entidad', 'Tipo_Entidad']]  # <- corregido

# 4. Guardamos con codificación UTF-8-SIG
df_entidades.to_csv("Dim_Entidad_Pro.csv", index=False, encoding='utf-8-sig')

print("✅ Tabla 'Dim_Entidad_Pro.csv' generada.")
print(df_entidades.head())


✅ Tabla 'Dim_Entidad_Pro.csv' generada.
   Entidad_ID entidad            Tipo_Entidad
0           1   Alfin  Banca Múltiple / Otros
1           2  Azteca  Banca Múltiple / Otros
2           3    BBVA            Bancos Top 4
3           4     BCI  Banca Múltiple / Otros
4           5     BCP            Bancos Top 4
